# Pandas 요약!

## 00. 정의

[Pandas 공홈](https://pandas.pydata.org/about/)에 따르면 "데이터 분석 라이브러리".

#### 특징
- 통합 인덱싱을 지원해서 빠른 DataFrame 의 조작을 지원
- Excel 이나 CSV 데이터와 인메모리 데이터 RW 지원
- 유연한 데이터 조작 + 데이터 정렬 - 누락 데이터 처리
- 라벨 기반의 슬라이싱/펜시 인덱싱
- 데이터 구조(DataFrame  말하는 거 같음)에 컬럼의 추가 및 삭제 가능
- 강력한 merge, join, group by 지원
- 시계열 지원(time series)

TODO:pdb

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from pandas_datareader import data as web

## 01. 기본 - 시리즈와 데이터프레임

### 01-01. Series
- Pandas 데이터를 구성하는 1 차원 자료구조
- Index 가 존재 (중복 가능)
- 기본 Dictionary 와 호환

#### a. Series 찍어보기

In [11]:
series_test = pd.Series([2017, 2018, 2019, 2020])
# series_test  # 1 - 시리즈 자체
# series_test.values  # 2 - 값
series_test.index  # 3 - 인덱스

RangeIndex(start=0, stop=4, step=1)

##### b. Series - Index 조작

In [17]:
series_indice = pd.Series([2017, 2018, 2019, 2020], index=['c', 'a', 'b', 'b'])
series_indice['a']
series_indice[['a', 'c']]
series_indice['b']

b    2019
b    2020
dtype: int64

##### c. Series - 값의 정렬과 필터

In [22]:
series_sort = pd.Series([2017, 2018, 2019, 2020], index=['c', 'a', 'b', 'b'])

# series_sort

# series_sorted = series_sort.sort_index()  # 원본은 sorting 이 되어있지 않은 상태를 유지
# series_sorted

series_filtered = series_sort[series_sort > 2018]
series_filtered

b    2019
b    2020
dtype: int64

##### d. dictionary 호환

index 조작시 주의점

- 생성할 때 value 와 index 의 수가 맞지 않으면 `NaN` 삽입(dtype 과 무관하게)
- 생성 후 인덱스 주입 시 value 와 index 수 맞지 않으면 에러
- 생성 후 인덱스 주입 시 index 덮어씀

In [48]:
data_dict = { 'c' : 2021, 'a' : 2018, 'b' : 2019, 'b' : 2020 }
series_dict = pd.Series(data_dict)

# series_dict  # dictionary 는 중복키가 허용되지 않음 - value 기준으로 자동 정렬
# print(f"c in : {'c' in series_dict}, d in : {'d' in series_dict}")

# 객체 명과 인덱스 명 지정 
# series_dict.name = 'years'
# series_dict.index.name = 'idx'
# print(f"Series name : {series_dict.name}, Series index name : {series_dict.index.name}")
# series_dict

# 원하는 키로 정렬
indice = ['b', 'c', 'a', 'd']
series_dict = pd.Series(data_dict, indice)
# series_dict.index = indice[:-1]
series_dict
# series_dict.isnull()  # series_dict.notnull()

b    2020.0
c    2021.0
a    2018.0
d       NaN
dtype: float64

### 01-02. DataFrame

- 테이블 형식의 자료구조

##### a. DataFrame 찍어보기

In [56]:
dic = {'state' : ['Ohio','Ohio','Ohio','Nevada','Nevada','Nevada'], 
      'year' : [2000, 2001, 2002, 2001, 2002, 2003],
      'pop' : [1.5, 1.7, 3.6, 2.4, 2.9, 3.7]}
df = pd.DataFrame(dic)
# df = pd.DataFrame(dic, columns=['year', 'pop', 'status'])  # 칼럼 순서 변경
df

,year,pop,status
0,2000,1.5,NaN
1,2001,1.7,NaN
2,2002,3.6,NaN
3,2001,2.4,NaN
4,2002,2.9,NaN
5,2003,3.7,NaN


In [66]:
arr = [5 - i for i in range(6)]  # 5 ... 0 인덱스 부여
df.index = arr
df

,year,pop,status
5,2000,1.5,NaN
4,2001,1.7,NaN
3,2002,3.6,NaN
2,2001,2.4,NaN
1,2002,2.9,NaN
0,2003,3.7,NaN


##### b. column 또는 row 에 접근하기

In [ ]:
df = pd.read_csv('df_1.csv')
df.index = [i for i in range(len(df))]

# Column
df.year  # df['year']

# Row
# df.loc['0']  # Key Error
df.loc[0]

##### c. 칼럼의 추가와 삭제 / 테이블 전치

In [86]:
df = pd.read_csv('df_1.csv')

# 추가 
df['fresh'] = df['year'] > 2000
# df.fresh = df['year'] > 2000  # df.fresh 는 안됨
df

# 삭제 
del df['fresh']
df

# 전치
transformed = df.T  # 전치된 새로운 DataFrame 반환 
transformed
# df

,0,1,2,3,4,5
state,Ohio,Ohio,Ohio,Nevada,Nevada,Nevada
year,2000,2001,2002,2001,2002,2003
pop,1.5,1.7,3.6,2.4,2.9,3.2


##### d. 중첩 사전

In [92]:
nested_d = {'Nevada' : {2001: 2.4, 2002: 2.9},
        'Ohio' : {2000: 1.5, 2001: 1.7, 2002: 3.6}
       }

ndf = pd.DataFrame(nested_d) 
ndf

# 3중첩 가능 ?
# nested_3d = {'Nevada' : {2001: { 'pc' : True, 'mobile' : False }, 2002: { 'pc' : True, 'mobile' : False }},
#         'Ohio' : {2000: { 'pc' : True, 'mobile' : False }, 2001: { 'pc' : True, 'mobile' : False }, 2002: { 'pc' : True, 'mobile' : False }}
#        }

# n3df = pd.DataFrame(nested_3d) 
# n3df

ndf.columns.name = 'x'
ndf.index.name = 'y'
ndf

x,Nevada,Ohio
y,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


## 02. 색인

index 
- 각 로우와 칼럼에 대한 이름을 다른 메타 데이터와 같이 저장하는 객체
- 변경 불가능(?)

In [99]:
df = pd.read_csv('df_1.csv')
df.index = [i for i in range(len(df))]
index = df.index
# index[1] = 3  # TypeError: Index does not support mutable operations
df.index = [ chr(65 + i) for i in range(len(df))]  # 전체 뒤집어 쓰는 건 됨 
df

,state,year,pop
A,Ohio,2000,1.5
B,Ohio,2001,1.7
C,Ohio,2002,3.6
D,Nevada,2001,2.4
E,Nevada,2002,2.9
F,Nevada,2003,3.2


##### a. 색인 메서드
- insert : 색인 추가
- append : 색인 추가(맨뒤)
- isin : 존재 확인
- delete : i 위치 색인 삭제
- drop : 지정 값이 삭제된 색인 반환
- unique : 중복 삭제

Q. 색인을 변경해도 DataFrame 은 변경되지 않는다. 색인만 변경해서 사용하는 경우가 있을까?

In [127]:
hawaii = {'state' : 'Hawaii', 'pop' : 1.6, 'year': 1920}
df = pd.read_csv('df_1.csv')
df.index = [i for i in range(len(df))]

# inserted_index = df.index.insert(3, 3)  # index 만 추가되고 값은 없음
# inserted_index.unique()

# df.index.append(pd.Index[9])  # 안됨... Help!!!

# df.index.isin([3])  # 배열 넘겨야 함

# df.index.delete(3)  # delete 는 인덱스를 전달하고
# df.index.drop(3)    # drop 은 레이블(값)을 전달함 


Int64Index([0, 1, 2, 4, 5], dtype='int64')

## 03. 객체(Series, DataFrame) 메소드

#### 데이터 조작
- reindex
- drop
- 슬라이싱(loc/iloc)
- 사칙연산(add/sub/mul/div)
- 함수적용(apply/applymap)
- 정렬(sort_index/sort_values)
- rank + is_unique

##### a. reindex

In [3]:
dic = {'state' : ['Ohio','Ohio','Ohio','Nevada','Nevada','Nevada'], 
      'year' : [2000, 2001, 2002, 2001, 2002, 2003],
      'pop' : [1.5, 1.7, 3.6, 2.4, 2.9, 3.7]}
idx = [chr(i + 65) for i in range(6)]

df = pd.DataFrame(dic, idx)

df2 = df.reindex(['G'] + idx)
df2

# method bfill - 다음 유효 관측치, ffill, nearest - 가장 가까운 유효 관측치 
df3 = df.reindex(['G'] + idx, method='ffill')
df3


# 칼럼 기준으로
df4 = df.reindex(columns=['pop', 'year', 'state'])
df4


# ===== Help!!! bfill 이나 nearest 용례 찾기가 힘듦
# s_data = [2,4,6,8,10]
# s_idx = [chr(i + 65) for i in range(len(series))]
# series = pd.Series(s_data, s_idx)
# series

# # method bfill, ffill, nearest
# s1 = series.reindex(['G'] + idx, method='ffill')
# s1

# s2 = series.reindex(['A', 'B', 'G', 'C', 'D'], method='nearest')
# s2

,pop,year,state
A,1.5,2000,Ohio
B,1.7,2001,Ohio
C,3.6,2002,Ohio
D,2.4,2001,Nevada
E,2.9,2002,Nevada
F,3.7,2003,Nevada


##### b. drop
drop 의 기본 axis 는 0 이고 index 를 의미합니다. (`1` or `columns`)


In [4]:
df = pd.DataFrame(dic, idx)
df.drop(['A'])  # 원본과 무관


df.drop(['year', 'pop'], axis=1, inplace=True)  # 원본 삭제 
df

,state
A,Ohio
B,Ohio
C,Ohio
D,Nevada
E,Nevada
F,Nevada


##### c. 슬라이싱

슬라이싱과 대입, loc 과 iloc

In [5]:
df = pd.DataFrame(dic, idx)
# df[2:4]  # 2,3
df['A':'D']  # 연결 문자열에 대한 슬라이싱 - 마지막 포함 
# df[df.index > 'C']  # 문자열에 대한 산술 비교

# 01. 맞지 않는 값으로 치환 (broadcasting)
# df['A':'B'] = 5  
# df

# 02. 맞는 유형으로 치환
df['A':'B'] = {'state' : 'Seoul', 'pop' : 9.9, 'year' : 2020}
df

sr1 = pd.Series([i for i in range(len(idx))], idx)
sr1[[0, 3]]  #  값이 0 이거나 3인...
sr1[sr1.isin([0,3])]  # 다른 표현, 같은 결과 

A    0
D    3
dtype: int64

loc 과 iloc 은 지정 범위의 데이터를 반환한다.
2 차 배열이라고 가정했을 때, 첫째 인자는 로우의 위치 정보이고, 둘쨰 인자는 칼럼의 위치 정보이다

- loc 은 마지막을 포함한다
- iloc 은 마지막을 포함하지 않는다

In [6]:
df = pd.DataFrame(dic, idx)

# df.loc[['B','E'], 'state']
# df.iloc[[1, 4], 0]

df.loc[:'E', 'state']  # 첫번째가 [: 'E'] 가 아님에 유의
# df.iloc[:,:3][df.pop > 2]  # 이건 안되고
df.iloc[:,:3][df['pop'] > 2]  # 이건 됨

,state,year,pop
C,Ohio,2002,3.6
D,Nevada,2001,2.4
E,Nevada,2002,2.9
F,Nevada,2003,3.7


##### d. 사칙 연산

- 두개의 DataSet 을 연산할 때 한쪽이 NaN 이면 무조건 NaN 이됨(fill_value)
- `+` : add / radd
- `-` : sub / rsub
- axis 속성

In [7]:
df1 = pd.DataFrame(np.arange(12).reshape(3,4), columns=list('abcd'))
df1

df2 = pd.DataFrame(np.arange(20).reshape(4,5), columns=list('abcde'))
df2

df1 + df2

# df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


Series 와 DataFrame 간의 연산

In [8]:
series_col = pd.Series([i for i in range(5)])
series_col

# df1.add(series_col, axis=0)  # 칼럼에 더하기 

# series_row = pd.Series([i for i in range(4)], index=list('abcd'))
# df1.add(series_row)  # 로우에 더하기 

0    0
1    1
2    2
3    3
4    4
dtype: int64

##### e. 함수적용
참고 : [http://www.leejungmin.org/post/2018/04/21/pandas_apply_and_map/](http://www.leejungmin.org/post/2018/04/21/pandas_apply_and_map/)

- map : Series
- apply / applymap : DataFrame

In [9]:
f_df = pd.DataFrame(np.arange(20).reshape(4,5), index=list('abcd'))
f_df

lmbd = lambda x : x**2
avg_f = lambda x : x.mean()

f_df.apply(lmbd)  # 전체에 제곱. applymap 과 동일한 결과
f_df.apply(avg_f, axis=1)  # 지정 축에 대해서 평균으로 변환  

a     2.0
b     7.0
c    12.0
d    17.0
dtype: float64

##### f. 정렬

- sort_index
- sort_values

In [10]:
df

df.sort_index(ascending=False, axis=0)  # index 반대방향으로

df.sort_values(ascending=False, by='year')  # 값 기준 정렬
df.sort_values(ascending=False, by=['year', 'pop'])  # 값 기준 내 정렬

,state,year,pop
F,Nevada,2003,3.7
C,Ohio,2002,3.6
E,Nevada,2002,2.9
D,Nevada,2001,2.4
B,Ohio,2001,1.7
A,Ohio,2000,1.5


##### g. rank + is_unique

In [11]:
# df.rank()  # 동률은 평균
# df['year'].is_unique
# df.rank(method='min')  # dense_rank 
# df.rank(method='max')  # dense_rank 
# df.rank(method='first')  # dense_rank => numeric data 에서만 유효!
df.rank(method='dense')  # dense_rank 

,state,year,pop
A,2.0,1.0,1.0
B,2.0,2.0,2.0
C,2.0,3.0,5.0
D,1.0,2.0,3.0
E,1.0,3.0,4.0
F,1.0,4.0,6.0


## 04. 통계 요약


#### 데이터 통계
- sum | mean |idxmin | idxmax | cumsum 
- describe

In [12]:
df

,state,year,pop
A,Ohio,2000,1.5
B,Ohio,2001,1.7
C,Ohio,2002,3.6
D,Nevada,2001,2.4
E,Nevada,2002,2.9
F,Nevada,2003,3.7


In [13]:
df['year'].min()
df['year'].max()
df['pop'].mean()

df_num = pd.DataFrame(np.arange(20).reshape(4,5))
# df_num.max(axis=0)  # 마지막 로우
# df_num.max(axis=1)  # 마지막 칼럼

df_num.cumsum()  # 누산 

,0,1,2,3,4
0,0,1,2,3,4
1,5,7,9,11,13
2,15,18,21,24,27
3,30,34,38,42,46


In [14]:
df.describe()

,year,pop
count,6.000000,6.000000
mean,2001.500000,2.633333
std,1.048809,0.933095
min,2000.000000,1.500000
25%,2001.000000,1.875000
50%,2001.500000,2.650000
75%,2002.000000,3.425000
max,2003.000000,3.700000


## 05. 상관관계 공분산

~`conda install panads-datareader` 를 실행해 주세요.~

`pip install pandas-datareader` 로 해야 올바른 설치가 됩니다.([공홈](https://pypi.org/project/pandas-datareader/))

##### 공분산(Covariance)
두 확률 변수 사이에 퍼져있는 정도
- cov(x1, x2) > 0 : x1 과 x2 는 정비례
- cov(x1, x2) < 0 : x1 과 x2 는 반비례
- cov(x1, x2) = 0 : x1 과 x2 는 독립적일 가능성이 높음

##### 상관관계(Correlation)
공분산에서 x1 과 x2 의 절대 수량(단위의 크기)를 보완한 개념 

In [17]:
all_data = {ticker: web.get_data_yahoo(ticker) for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

price = pd.DataFrame({ticker: data['Adj Close'] for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume'] for ticker,data in all_data.items()})

In [19]:
price.head()
volume.head()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2015-07-27,44455500.0,3706200.0,39701400.0,2675400
2015-07-28,33618100.0,2721000.0,34328900.0,1727300
2015-07-29,37011700.0,3378400.0,40945900.0,1575100
2015-07-30,33628300.0,1994700.0,39777900.0,1474200
2015-07-31,42885000.0,3580200.0,31201500.0,1706100


In [21]:
variance = price.pct_change()  # 퍼센티지의 변화율. 데이터가 4개라면 pct_change 는 3개입니다. 
variance.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2020-07-20,0.021074,0.010071,0.042981,0.033103
2020-07-21,-0.013802,-0.002453,-0.013469,-0.004662
2020-07-22,0.002809,0.020704,0.014371,0.006462
2020-07-23,-0.045516,-0.010414,-0.043495,-0.033669
2020-07-24,-0.002477,-0.012095,-0.006122,-0.002514


In [24]:
variance['MSFT'].corr(variance['AAPL'])  # MS 와 Apple 가격 상관관계 => 1에 가까우니까 MS 가 오르면 AAPL 도 오를 확률이 높다 

0.7167219277418521

In [25]:
variance['MSFT'].cov(variance['AAPL'])  # TODO: 물어보기. MS 와 Apple 가격 공분산 => 비슷해야 되는 거 아닌가...

0.00022814183451327298

In [26]:
variance.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.526898,0.716722,0.671977
IBM,0.526898,1.000000,0.593071,0.543362
MSFT,0.716722,0.593071,1.000000,0.785881
GOOG,0.671977,0.543362,0.785881,1.000000


In [29]:
variance.corrwith(volume)  # 퍼센트 변화율에 따른 시가 총액의 비교

AAPL   -0.133555
IBM    -0.103590
MSFT   -0.066203
GOOG   -0.142452
dtype: float64

## 06. 유일값, 빈도

- unique
- value_counts
- match

In [41]:
s1 = pd.Series([0,1,2,3,3], list('abbde'))
s1.unique()
s1.index.unique()

# 오직 Series 에서만!
# df1 = pd.DataFrame(np.arange(20).reshape(4,5), list('abcd'))
# df1[0]['a']=10
# df1.unique()

Index(['a', 'b', 'd', 'e'], dtype='object')

In [44]:
s1.value_counts(sort=False)  # sort 는 asc / desc

0    1
1    1
2    1
3    2
dtype: int64

In [47]:
df['state'].str.match(pat='^O')  # document 에 Series.match 는 없고 Series.str.match 만 있음

A     True
B     True
C     True
D    False
E    False
F    False
Name: state, dtype: bool

### 종합



In [88]:
import plotly.graph_objs as go

hist = pd.DataFrame({
    'Q1': [1,3,4,3,4],
    'Q2': [2,3,1,2,3],
    'Q3': [1,5,2,4,4],
    'Q4': [3,8,4,2,5]
})

frq = hist.apply(pd.value_counts).fillna(0)
frq.index = [chr(65 + i) for i in range(len(frq))]
frq

,Q1,Q2,Q3,Q4
A,1.0,1.0,1.0,0.0
B,0.0,2.0,1.0,1.0
C,2.0,2.0,0.0,1.0
D,2.0,0.0,2.0,1.0
E,0.0,0.0,1.0,1.0
F,0.0,0.0,0.0,1.0
